In [39]:
!pip install pandas
!pip install ipynb
!pip install sklearn
!pip install pyemma
!pip install hmmlearn
!pip install import-ipynb
import import_ipynb
import pandas as pd
import random as rd
import sys

import ipynb.fs.full.kai_data_visualization as kai
import ipynb.fs.full.adyang as andrew


import matplotlib
import matplotlib.dates as md
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.covariance import EllipticEnvelope
from pyemma import msm
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM
from hmmlearn import hmm
import numpy as np

from datetime import datetime

np.random.seed(42)

In [40]:
# code cleaning

def cleanData(fileName = '../data/gemini_BTCUSD_2020_1min.csv', timeRangeInHour=1, valueUsed="open"):
    x_filename = '../data/gemini_BTCUSD_2021_1min.csv'
    x_testFile = '../data/gemini_BTCUSD_2020_1min.csv'
    # Read Data
    df = pd.read_csv(x_filename, skiprows=[0], low_memory=False)
    cdf = df.drop(columns=['Date', 'Symbol', 'Unix Timestamp'], axis=1)
    return cdf

In [41]:
cleandf = cleanData()
def random(bit_num, data, cleandf):
#     print(cleandf)
    open_col = cleandf[data]
#     print(open_col)
#     print(open_col[3])
    num_choice = rd.randint(0, len(open_col))
#     print(num_choice)
    random_choice = rd.choices(open_col, k = num_choice)
#     print(random_choice)
    # buy in first
    result = 0
    for i in range(num_choice-1):
        result += (bit_num*(random_choice[i+1]-random_choice[i]) )
        i += 1

#         print(result)
    print("final result: ", result)
    return result

In [42]:
random(10, "Open", cleandf)

final result:  122252.60000000107


122252.60000000107

In [43]:
# train markov model to get transition matrix
def getTransitionMatrix (df):
    df = np.array(df)
    model = msm.estimate_markov_model(df, 1)
    return model.transition_matrix


In [44]:
# def algoPredict2(remodel, testdf, buyInState, buyOutState, num_state=3, valueUsed="Open", stockPerBuy=10, initialMoney=0, allowDebt=True):

    
    
#     # Note: initialMoney and allowDebt is not used at the moment
#     Z2 = remodel.predict(testdf)
    
#     canBuy = True
#     money = 0
#     inHandStock = 0
#     for i in range(len(Z2)):
#         if i == (len(Z2) - 1) and inHandStock > 0:
#             print("clearing out everything at the end")
#             print(f'before last trade, money = {money} and final stock left = {inHandStock}')
#             money += inHandStock * testdf.at[i,valueUsed]
#             inHandStock = 0
#         elif Z2[i] == buyInState and canBuy:
#             inHandStock += stockPerBuy
#             money -= stockPerBuy * testdf.at[i,valueUsed]
#             canBuy = False
#         elif Z2[i] == buyOutState and inHandStock > 0 and not canBuy:
#             inHandStock -= stockPerBuy
#             money += stockPerBuy * testdf.at[i,valueUsed]
#             canBuy = True
            
#     print(f'Total resulted money from this trading session = {money}')
#     return money

In [45]:
def overall(data, bit_coin_num, state_num):
#     random_result = random(bit_coin_num, data)
    
    # markov
    
    chunkSize = kai.testdf.size/365
    intervals = np.array_split(kai.testdf, chunkSize)
    index = 0
    for interval in intervals:
        predict_result = kai.algoPredict2(kai.remodel, interval, 0, 1, num_state = state_num, valueUsed = data)
        random_result = random(bit_coin_num, data, interval)
        max_result = andrew.theoretical_max_gain_no_overlap(interval, 10, price_type=data)
        score = (predict_result-random_result)/(max_result-random_result)
        print("==========", index)
        print(score)
        index += 1


    
    
#     predict_result = kai.algoPredict2(kai.remodel, kai.testdf, 0, 1, num_state = state_num, valueUsed = data)
#     print("hhhhhh")
    
#     print("random: ", random_result, "markov: ", predict_result)
    
    
#     # max
#     max_result = andrew.theoretical_max_gain_no_overlap(andrew.raw_data, datetime(year=2020, month=1, day=1), datetime(year=2020, month=12, day=31), 10, price_type=data)
    
    
#     score = (predict_result-random_result)/(max_result-random_result)
    
#     print("score: ", score)

In [46]:
kai.algoPredict2(kai.remodel, kai.testdf, 0, 1)

Total resulted money from this trading session = 381422.3000000033


381422.3000000033

In [47]:
overall("Open", 10, 3)

Total resulted money from this trading session = -903.5999999999767
final result:  172.20000000001164
        Open      High       Low     Close    Volume
0   36467.99  36467.99  36404.75  36443.88  0.315436
1   36753.43  36753.43  36441.79  36467.99  2.688382
2   36663.07  36770.74  36663.07  36753.43  0.035037
3   36704.30  36704.30  36641.22  36663.07  0.377833
4   36622.96  36718.52  36622.96  36704.30  0.045218
..       ...       ...       ...       ...       ...
69  36837.48  36847.71  36828.73  36847.71  0.016451
70  36841.44  36841.44  36821.09  36837.48  1.822248
71  36784.52  36841.44  36777.98  36841.44  0.156711
72  36753.56  36784.52  36750.76  36784.52  0.033085
73  36796.99  36796.99  36735.09  36753.56  0.990870

[74 rows x 5 columns]


KeyError: "['Date' 'Symbol' 'Unix Timestamp'] not found in axis"